Module : <b>Blockchain</b><br> 
Travail réalisé par :<b>Mehdi AMOR OUAHMED</b><br>
<br> Niveau : <b>3 DSIA</b>  <br> Année Universitaire : <b>2023-2024</b>
# Intéragir avec un smart contrat<br>

In [1]:
from web3 import Web3
import os
from dotenv import find_dotenv, load_dotenv
import json

In [2]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [3]:
infura_url = "https://data-seed-prebsc-1-s1.binance.org:8545"
w3 = Web3(Web3.HTTPProvider(infura_url))

if w3.is_connected():
    print("Connected to BSC Testnet")
else:
    print("Connection failed")

Connected to BSC Testnet


In [5]:
f = open(os.path.join('src', 'abi.json'))
contract_abi = json.load(f)
contract_address = "0x1881ef0Eb82142873f6751B76C5400D08369694f"

contract = w3.eth.contract(address=contract_address, abi=contract_abi)

In [6]:
contract.functions.nombre_taches_accomplies().call()

0

In [7]:
contract.functions.voir_progres().call()

[[], []]

In [8]:
wallet = "0x0be5E8Df43124d3B94e88fF0e6C866f1A4891893"
private_key = os.getenv("private_key")

In [9]:
add_task_transaction = contract.functions.ajouter_une_tache('Faire le DM du modeule Blockchain').build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

In [10]:
signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

In [11]:
transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [12]:
contract.functions.voir_progres().call()

[['Faire le DM du modeule Blockchain'], [False]]

In [13]:
add_task_transaction = contract.functions.tache_accomplie('Faire le DM du modeule Blockchain').build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [14]:
contract.functions.voir_progres().call()

[['Faire le DM du modeule Blockchain'], [True]]

In [15]:
contract.functions.pourcentage_accompli().call()

100

In [16]:
add_task_transaction = contract.functions.reset().build_transaction(
    {"gasPrice": w3.eth.gas_price, 
     "chainId": w3.eth.chain_id, 
     "from": wallet, 
     "nonce": w3.eth.get_transaction_count(wallet)})

signed_transaction = w3.eth.account.sign_transaction(add_task_transaction, private_key=private_key)

transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

In [17]:
contract.functions.voir_progres().call()

[[], []]